In [13]:
import json

with open('./data/alpaca_data_cleaned.json', 'r') as f:
    data = json.load(f)


In [17]:
data = [x for x in data if x['input'] == '']
len(data)

32603

In [21]:
d1 = data[:5000]
d2 = data[5000:10000]
d3 = data[10000:15000]
d4 = data[15000:20000]
d5 = data[20000:25000]
d6 = data[25000:]

In [28]:
with open('output6.txt', 'w') as f:
    for d in d6:
        if d['input'] == '':
            f.write(d['instruction'])
            f.write('\n\n')
            f.write(d['output'])
            f.write('\n---\n')


In [34]:
import docx

def read_docx(file_path):
    doc = docx.Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

def save_to_txt(file_path, content):
    with open(file_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(content)

# Example usage:
input_docx_path = 'output6 (1).docx'
output_txt_path = 'out6.txt'

persian_text = read_docx(input_docx_path)
save_to_txt(output_txt_path, persian_text)


In [16]:
import json


output = []
for i in range(6):
    file = f'./data/out{i+1}.txt'
    with open(file, 'r') as f:
        data = f.read()

    data = data.split('---')
    for d in data:
        tmp = d.split('\n\n')
        question = tmp[0]
        answer = ''.join(tmp[1:])
        output.append({'text': f'### Human: {question} ### Assistant: {answer}'})

with open('data.jsonl', 'w') as f:
    for o in output:
        json.dump(o, f, ensure_ascii=False)
        f.write('\n')

In [17]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset('sinarashidi/alpaca-persian')

# Shuffle the dataset and slice it
# dataset = dataset['train'].shuffle(seed=42).select(range(1000))

# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': '\\\n'.join(reformatted_segments)}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)


/home/sina/projects/agp/llama2-finetune/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
HF google storage unreachable. Downloading and preparing it from source
Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 58.84it/s]
Generating train split: 35117 examples [00:00, 78714.00 examples/s] 
Map: 100%|██████████| 35117/35117 [00:03<00:00, 11452.02 examples/s]


In [20]:
transformed_dataset.push_to_hub("sinarashidi/alpaca-persian-llama2")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.62s/it]


In [28]:
import itertools 


itertools.accumulate()

[1, 3, 6, 10, 15]